In [ ]:
import gc   #garbage collector-to free memory
gc.enable() #Activate
import zipfile #to unzip files
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
!pip install --upgrade kaggle
from google.colab import files
from datetime import datetime
api_token=files.upload()

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=5281ba7db82bfb08a0597bcbe3a38422ed910135bcae5adb1ae235bbacd4efde
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


Saving kaggle.json to kaggle.json


In [ ]:
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c instacart-market-basket-analysis

 99% 195M/196M [00:01<00:00, 216MB/s]
100% 196M/196M [00:01<00:00, 198MB/s]


In [ ]:
with zipfile.ZipFile('/content/instacart-market-basket-analysis.zip','r')as z:
  z.extractall('.')

In [ ]:
with zipfile.ZipFile('/content/orders.csv.zip','r')as z:
    z.extractall('.')

In [ ]:
orders=pd.read_csv('./orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
import numpy as np
orders['days_since_prior_order'].replace(to_replace=np.nan,value=0,inplace=True)
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,0.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
#selecting only train data
te_orders=orders[orders.eval_set=='test']
te_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Merging data frame with users'feature**

In [ ]:
user=pd.read_csv('/content/drive/MyDrive/Instacart/user.csv')
user.drop('Unnamed: 0',axis=1,inplace=True)
user.head()

,user_id,Total orders,user_re_ratio,user_ntre_ratio
0,1,10,0.694915,0.305085
1,2,14,0.476923,0.523077
2,3,12,0.625000,0.375000
3,4,5,0.055556,0.944444
4,5,4,0.378378,0.621622


In [ ]:
te_orders=te_orders.merge(user,on='user_id',how='left')
te_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio
0,2774568,3,test,13,5,15,11.0,12,0.625000,0.375000
1,329954,4,test,6,3,12,30.0,5,0.055556,0.944444
2,1528013,6,test,4,3,16,22.0,3,0.142857,0.857143
3,1376945,11,test,8,6,11,8.0,7,0.351064,0.648936
4,1356845,12,test,6,1,20,30.0,5,0.175676,0.824324


**Merging data frame with user-products'feature**

Here products are prior ordered products of a user

In [ ]:
user_prd=pd.read_csv('/content/drive/MyDrive/Instacart/uxp.csv')
user_prd.drop('Unnamed: 0',axis=1,inplace=True)
user_prd.head()

,user_id,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5
0,1,196,1.000000,9.0,5.0
1,1,10258,1.000000,7.0,5.0
2,1,10326,0.166667,0.0,0.0
3,1,12427,1.000000,8.0,5.0
4,1,13032,0.333333,0.0,2.0


In [ ]:
te_orders=te_orders.merge(user_prd,on=['user_id'],how='left')
te_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5
0,2774568,3,test,13,5,15,11.0,12,0.625,0.375,248,0.090909,1.0,0.0
1,2774568,3,test,13,5,15,11.0,12,0.625,0.375,1005,0.333333,0.0,1.0
2,2774568,3,test,13,5,15,11.0,12,0.625,0.375,1819,0.333333,2.0,0.0
3,2774568,3,test,13,5,15,11.0,12,0.625,0.375,7503,0.100000,0.0,0.0
4,2774568,3,test,13,5,15,11.0,12,0.625,0.375,8021,0.090909,0.0,0.0


**Merging data frame with products' feature**

In [ ]:
prd=pd.read_csv('/content/drive/MyDrive/Instacart/prd.csv')
prd.drop('Unnamed: 0',axis=1,inplace=True)
prd.head()

,product_id,prd_cart_last_5,prd_ntre_ratio
0,1,5.4,0.386609
1,2,11.6,0.866667
2,3,4.4,0.267148
3,4,16.4,0.553191
4,5,8.2,0.400000


In [ ]:
te_orders=te_orders.merge(prd,on='product_id',how='left')
te_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio
0,2774568,3,test,13,5,15,11.0,12,0.625,0.375,248,0.090909,1.0,0.0,9.2,0.599749
1,2774568,3,test,13,5,15,11.0,12,0.625,0.375,1005,0.333333,0.0,1.0,15.8,0.559395
2,2774568,3,test,13,5,15,11.0,12,0.625,0.375,1819,0.333333,2.0,0.0,8.4,0.507838
3,2774568,3,test,13,5,15,11.0,12,0.625,0.375,7503,0.100000,0.0,0.0,6.6,0.446449
4,2774568,3,test,13,5,15,11.0,12,0.625,0.375,8021,0.090909,0.0,0.0,5.4,0.408843


**Merging data frame with dow feature**

In [ ]:
user_dow=pd.read_csv('/content/drive/MyDrive/Instacart/uxd.csv')
user_dow.drop('Unnamed: 0',axis=1,inplace=True)
user_dow.head()

,user_id,order_dow,dow_re_ratio,dow_ntre_ratio
0,1,1,0.882353,0.396157
1,1,2,0.444444,0.410229
2,1,3,0.545455,0.413728
3,1,4,0.727273,0.409021
4,2,1,0.671875,0.396157


In [ ]:
te_ord=te_orders.merge(user_dow,on=['user_id','order_dow'],how='left')
te_ord.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio,dow_re_ratio,dow_ntre_ratio
0,2774568,3,test,13,5,15,11.0,12,0.625,0.375,248,0.090909,1.0,0.0,9.2,0.599749,NaN,NaN
1,2774568,3,test,13,5,15,11.0,12,0.625,0.375,1005,0.333333,0.0,1.0,15.8,0.559395,NaN,NaN
2,2774568,3,test,13,5,15,11.0,12,0.625,0.375,1819,0.333333,2.0,0.0,8.4,0.507838,NaN,NaN
3,2774568,3,test,13,5,15,11.0,12,0.625,0.375,7503,0.100000,0.0,0.0,6.6,0.446449,NaN,NaN
4,2774568,3,test,13,5,15,11.0,12,0.625,0.375,8021,0.090909,0.0,0.0,5.4,0.408843,NaN,NaN


In [ ]:
te_ord.drop(['eval_set','order_number','order_dow'],axis=1,inplace=True)
te_ord=te_ord.fillna(0)
te_ord.head()

,user_id,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio,dow_re_ratio,dow_ntre_ratio,order_id
0,3,15,11.0,12,0.625,0.375,248,0.090909,1.0,0.0,9.2,0.599749,0.0,0.0,2774568
1,3,15,11.0,12,0.625,0.375,1005,0.333333,0.0,1.0,15.8,0.559395,0.0,0.0,2774568
2,3,15,11.0,12,0.625,0.375,1819,0.333333,2.0,0.0,8.4,0.507838,0.0,0.0,2774568
3,3,15,11.0,12,0.625,0.375,7503,0.100000,0.0,0.0,6.6,0.446449,0.0,0.0,2774568
4,3,15,11.0,12,0.625,0.375,8021,0.090909,0.0,0.0,5.4,0.408843,0.0,0.0,2774568


In [ ]:
te_ord.to_csv('/content/drive/MyDrive/Instacart/test_final.csv')

As we have seen in EDA that each user_id has only 1 test order_id (proved below also)  

Here in above df we hv two composite keys user_id+product_id and order_id+product_id so, we can remove one .

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
ord_id=orders[orders.eval_set=='test'][['user_id','order_id']]
ord_id.head()

,user_id,order_id
38,3,2774568
44,4,329954
53,6,1528013
96,11,1376945
102,12,1356845


In [ ]:
len(ord_id['user_id'].unique())

75000

In [ ]:
len(ord_id['order_id'].unique())

75000

In [ ]:
ord_id.shape

(75000, 2)

this can be clearly noticed here also that each user_id has only 1 test order if any.

In [ ]:
te_ord.drop(['order_id'],axis=1,inplace=True)
te_ord.head()

,user_id,order_hour_of_day,days_since_prior_order,Total orders,user_re_ratio,user_ntre_ratio,product_id,user_prd_re_ratio,times_cart_123,times_prd_last_5,prd_cart_last_5,prd_ntre_ratio,dow_re_ratio,dow_ntre_ratio
0,3,15,11.0,12,0.625,0.375,248,0.090909,1.0,0.0,9.2,0.599749,0.0,0.0
1,3,15,11.0,12,0.625,0.375,1005,0.333333,0.0,1.0,15.8,0.559395,0.0,0.0
2,3,15,11.0,12,0.625,0.375,1819,0.333333,2.0,0.0,8.4,0.507838,0.0,0.0
3,3,15,11.0,12,0.625,0.375,7503,0.100000,0.0,0.0,6.6,0.446449,0.0,0.0
4,3,15,11.0,12,0.625,0.375,8021,0.090909,0.0,0.0,5.4,0.408843,0.0,0.0


In [ ]:
te_ord.shape

(4833292, 13)

In [ ]:
te_ord.to_csv('/content/drive/MyDrive/Instacart/test.csv')